# Configuration
-------------------


Les cellules de cette section doivent être exécutées avant le reste du document. Si vous êtes déconnecté de l'engin d'exécution par Google Colab, il vous faudra exécuter cette section de nouveau.

Le code de cette section a pour fin customiser la commande magique `%%javascript` et ajouter quelques paramètres qui permettent que le code fourni soit exécuté côté conteneur Colab:


* `--target=[ node | browser | disk ]` :
  - `node`: indique que le code JavaScript fourni dans la cellule sera sauvegardé dans un fichier `.js` et exécuté par Node.js.
  - `browser`: la valeur browser correspond au comportement défaut de la commande `%%javascript`
  - `disk`: le contenu de la cellule sera seulement stocké dans le fichier `.js`.

* `--filename=FILENAME` : le nome du fichier qui sera créé sur le disque de la machine virtuelle

* `--port=PORT` : (optionnel) Node.js sera exécuté en *background* pour éviter que l'exécution de la cellule bloque le reste du cahier. Le numéro de port fourni sera exporté dans la variable d'environnement `NODE_PORT`.

In [ ]:
"""
Le code de cette cellule ira créer quelques dossiers
pour mieux organiser les fichiers JavaScript du cours, installer
les extensions et mettre à jour la version de Node.js
utilisée par Colab.
"""
directories = (
    'json-mock-api', 'express-mock-api'
    )
for dir in directories:
  !mkdir -p /content/$dir

!pip3 install classroom-extensions
%load_ext classroom_extensions.node_install
%install_nodejs
%load_ext classroom_extensions.web

# API REST (Representational State Transfer)
---------

* Une [API REST](https://www.redhat.com/fr/topics/api/what-is-a-rest-api) (également appelée API RESTful) est une interface de programmation d'application qui respecte les contraintes du style d'architecture REST et permet d'interagir avec les services web RESTful.

* L'architecture REST a été proposée par Roy Fielding à l'Université UC Irvine en tant que partie intégrante de [sa dissertation](https://www.ics.uci.edu/~fielding/pubs/dissertation/rest_arch_style.htm).

## REST

* REST n'est pas une norme ni un protocole, mais plutôt un ensemble d'architectures pour les API.

* Lorsqu'un client envoie une requête à une API RESTful, l'API crée une représentation de la ressource demandée et la rend accessible via un point d'accès (endpoint).

* La représentation, transmise via HTTP, est généralement au format JSON, bien que d'autres formats soient possibles.

* Les en-têtes HTTP et les paramètres d'URL sont essentiels pour les API REST, car ils contiennent des informations pour identifier les ressources et autoriser les utilisateurs, entre autres.

## Les principes de REST

Pour être considerée une API REST, une API doit respecter [les critères suivants](https://restfulapi.net/rest-architectural-constraints/):

1. **Architecture client-serveur** : Des ressources, des clients et des serveurs communiquent via HTTP, avec une séparation nette entre ces rôles. Les clients et les serveurs sont interchangeables et évoluent indépendamment tant que l'interface reste constante.

2. **Communication client-serveur sans état (*stateless*) :** Les données du client ne sont jamais stockées par le serveur, chaque requête `GET` est traitée indépendamment. Le serveur ne conserve aucun état de l'application client, et les informations d'état sont transmises par le client via les requêtes REST. Bien qu'un concept de *session* puisse exister, le client doit fournir les informations nécessaires pour identifier cette session au serveur. Le serveur est interchangeable et peut transférer l'état de la session vers d'autres serveurs ou systèmes, comme une base de données.

3. **Mise en cache des données :** La mise en cache, bien qu'utile pour améliorer les performances de la communication client-serveur, peut compromettre la cohérence des données. Afin d'éviter la mise en cache lorsque nécessaire, les réponses peuvent spécifier cette exigence, par exemple en utilisant des [en-têtes HTTP appropriés](https://developer.mozilla.org/fr/docs/Web/HTTP/Headers/Cache-Control).

4. **Système à couches :** La hiérarchisation des serveurs dans le service API est cruciale pour garantir que le client ne distingue pas une connexion directe d'une connexion via un nœud intermédiaire, tel qu'un *proxy* ou un pare-feu. Ce modèle à couches offre la possibilité de mettre en place une scalabilité, notamment par le biais d'un équilibreur de charge.

5. **Interface uniforme :** Une interface standardisée est essentielle pour faciliter la communication entre clients et serveurs. Les développeurs familiers avec votre API devraient pouvoir appliquer des approches similaires à d'autres APIs. Cela suppose de respecter notamment l'identification des ressources via l'URL, la manipulation des ressources via leurs représentations, l'utilisation de messages auto-descriptifs, l'adoption de formats standards tels que JSON ou XML, et la mise en œuvre des concepts HATEOAS (*Hypermedia As The Engine Of the Application State*).

6. **Code à la demande (facultatif) :** Il offre la flexibilité d'envoyer du code exécutable du serveur vers le client, permettant ainsi d'étendre les fonctionnalités du client en déplaçant la logique depuis le serveur.


### HATEOAS (*Hypermedia As The Engine Of the Application State*)

HATEOAS est une contrainte et un style architecturaux qui préconisent l'inclusion de liens hypermédias dans les réponses d'une API pour faciliter la navigation vers des ressources connexes en suivant les liens fournis.

Ce principe est similaire à la navigation web elle-même : une page contenant divers liens vers d'autres pages. Les liens hypermédias sont utilisés pour piloter l'état de l'application, plutôt que l'inverse.


**Exemple:**

Un appel au point de terminaison `https://api.example.com/users/123`, pour obtenir les details sur l'utilisateur dont l'identifiant est `123`, pourrait retourner:

```javascript
{
  "user": {
    "id": 123,
    "name": "Jean Dupont",
    "links": [
      {
        "rel": "self",
        "href": "https://api.example.com/users/123",
        "method": "GET"
      },
      {
        "rel": "posts",
        "href": "https://api.example.com/users/123/posts",
        "method": "GET"
      },
      {
        "rel": "friends",
        "href": "https://api.example.com/users/123/friends",
        "method": "GET"
      },
      {
        "rel": "new-post",
        "href": "https://api.example.com/users/123/posts",
        "method": "POST"
      }
    ]
  }
}

```

## Anatomie d'une API REST

**En pratique, qu'est-ce qu'une API REST ?** Nous avons déjà utilisé plusieurs APIs dans les cahiers précédents pour obtenir des informations sur des images, des lauréats du Prix Nobel, des planètes de la série *Star Wars*, et plus encore.
* Du point de vue de l'utilisation, une API REST se compose essentiellement de quatre éléments :
  - **Le point de terminaison** (également appelé *endpoint* en anglais).
  - **La méthode HTTP** utilisée pour effectuer une requête.
  - **Les en-têtes** des requêtes et des réponses HTTP.
  - **Les données** envoyées dans le corps des requêtes/réponses HTTP.

* Le point de terminaison correspond à l'URL par laquelle l'API est accessible. Par exemple, le point de terminaison racine de l'API [Star Wars](https://swapi.dev/api/) est: `https://swapi.dev/api` et celui de l'API de Twitter est: `https://api.twitter.com`

* Le chemin indiqué après le point de terminaison racine d'une API REST correspond à la ressource demandée. Par exemple, dans l'appel:
  ```
  GET https://swapi.dev/api/planets/1
  ```
  le chemin `planets/1` représente la ressource associée à la première planète de la liste de planètes de *Star Wars*.

* Pour déterminer le chemin à utiliser, il est nécessaire de consulter la documentation de l'API. Par exemple, [la documentation](https://swapi.dev/documentation#base) de l'API *Star Wars* explique que pour obtenir des informations sur un personnage spécifique, il faut utiliser la ressource `people` suivie de l'identifiant du personnage :
```
people/:id/ -- get a specific people resource
```
L'utilisation de `:` dans un chemin, comme illustré dans le routage d'Express, indique que `id` est une variable dont la valeur sera fournie lors de l'envoi de la requête.

* Une API REST peut également permettre aux utilisateurs de spécifier des paramètres pour des requêtes de recherche. Bien que les paramètres ne fassent pas nécessairement partie intégrante de toutes les APIs REST, ils sont fréquemment présents dans de nombreuses APIs. Dans l'[API Prix Nobel](https://www.nobelprize.org/about/developer-zone-2) que nous avons utilisé pour obtenir des informations sur les lauréats du Prix Nobel, nous avons utilisé des paramètres pour affiner nos recherches. Par exemple, nous avons utilisé :
```
http://api.nobelprize.org/2.1/laureates?name=Alice&birthCountry=Canada
```
pour obtenir la liste des lauréats nés au Canada et dont le nom contient `Alice`.

* Les paramètres de recherche suivent la ressource demandée et sont introduits par `?`, séparés par `&`.

## Tester les API REST avec `curl`

On peut utiliser des API disponibles dans divers langages pour créer un client HTTP qui envoie des requêtes à une API REST, comme nous l'avons vu avec l'API `fetch` dans les cahiers précédents.


Une option plus simple consiste à utiliser l'outil [curl](https://curl.se/), compatible avec Linux, MacOS et Windows, ou des outils tels que [Postman](https://www.postman.com/).


De nombreuses documentations d'API REST utilisent `curl` pour illustrer les appels aux ressources. Si vous maîtrisez `curl` pour tester une API, vous saurez l'utiliser pour tester d'autres API.


**Exemples :**

Les exemples ci-dessous vous montrent comment utiliser `curl` pour envoyer des requêtes et obtenir des informations de l'API *Star Wars*. Pour lancer une requête et récupérer des informations sur les vaisseaux spatiaux :


In [ ]:
!curl https://swapi.dev/api/starships/

{"count":36,"next":"https://swapi.dev/api/starships/?page=2","previous":null,"results":[{"name":"CR90 corvette","model":"CR90 corvette","manufacturer":"Corellian Engineering Corporation","cost_in_credits":"3500000","length":"150","max_atmosphering_speed":"950","crew":"30-165","passengers":"600","cargo_capacity":"3000000","consumables":"1 year","hyperdrive_rating":"2.0","MGLT":"60","starship_class":"corvette","pilots":[],"films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"created":"2014-12-10T14:20:33.369000Z","edited":"2014-12-20T21:23:49.867000Z","url":"https://swapi.dev/api/starships/2/"},{"name":"Star Destroyer","model":"Imperial I-class Star Destroyer","manufacturer":"Kuat Drive Yards","cost_in_credits":"150000000","length":"1,600","max_atmosphering_speed":"975","crew":"47,060","passengers":"n/a","cargo_capacity":"36000000","consumables":"2 years","hyperdrive_rating":"2.0","MGLT":"60","starship_class":"Star Destroyer","pilot

Le JSON retourné par l'API n'est pas formaté. Sur Linux, pour le formater on va utiliser `json_pp`. On ajoute aussi l'argument `-s` pour la commande `curl`  pour ignorer les informations de téléchargement du JSON:  

In [ ]:
!curl -s https://swapi.dev/api/starships/10/ | json_pp

{
   "MGLT" : "75",
   "cargo_capacity" : "100000",
   "consumables" : "2 months",
   "cost_in_credits" : "100000",
   "created" : "2014-12-10T16:59:45.094000Z",
   "crew" : "4",
   "edited" : "2014-12-20T21:23:49.880000Z",
   "films" : [
      "https://swapi.dev/api/films/1/",
      "https://swapi.dev/api/films/2/",
      "https://swapi.dev/api/films/3/"
   ],
   "hyperdrive_rating" : "0.5",
   "length" : "34.37",
   "manufacturer" : "Corellian Engineering Corporation",
   "max_atmosphering_speed" : "1050",
   "model" : "YT-1300 light freighter",
   "name" : "Millennium Falcon",
   "passengers" : "6",
   "pilots" : [
      "https://swapi.dev/api/people/13/",
      "https://swapi.dev/api/people/14/",
      "https://swapi.dev/api/people/25/",
      "https://swapi.dev/api/people/31/"
   ],
   "starship_class" : "Light freighter",
   "url" : "https://swapi.dev/api/starships/10/"
}


Pour afficher les en-têtes de requête et de réponse HTTP, on ajoute l'argument `-v` (*verbose*):

In [ ]:
!curl -s -v https://swapi.dev/api/starships/10/

*   Trying 52.58.110.120:443...
* Connected to swapi.dev (52.58.110.120) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
*  CAfile: /etc/ssl/certs/ca-certificates.crt
*  CApath: /etc/ssl/certs
* TLSv1.0 (OUT), TLS header, Certificate Status (22):
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.2 (IN), TLS header, Certificate Status (22):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.2 (IN), TLS header, Certificate Status (22):
* TLSv1.2 (IN), TLS handshake, Certificate (11):
* TLSv1.2 (IN), TLS header, Certificate Status (22):
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
* TLSv1.2 (IN), TLS header, Certificate Status (22):
* TLSv1.2 (IN), TLS handshake, Server finished (14):
* TLSv1.2 (OUT), TLS header, Certificate Status (22):
* TLSv1.2 (OUT), TLS handshake, Client key exchange (16):
* TLSv1.2 (OUT), TLS header, Finished (20):
* TLSv1.2 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.2 (OUT), TLS header, Certificate Status (22):
*

Si on s'intéresse qu'aux en-têtes de la réponse:

In [ ]:
!curl -s --head https://swapi.dev/api/starships/10/

HTTP/2 200 
server: nginx/1.16.1
date: Tue, 07 Nov 2023 00:14:39 GMT
content-type: application/json
vary: Accept, Cookie
x-frame-options: SAMEORIGIN
etag: "787ecb1fbae329ffafed92aec29ad8ac"
allow: GET, HEAD, OPTIONS
strict-transport-security: max-age=15768000



Nous verrons d'autres réglages possibles lorsque nous approfondirons notre étude des API REST.

## Les méthodes HTTP

La méthode HTTP, également désignée comme un **verbe**, correspond au type de requête HTTP envoyée au serveur, offrant un choix parmi les types suivants :

  - `GET`
  - `POST`
  - `PUT`
  - `PATCH`
  - `DELETE`

**Note:** Les méthodes `OPTIONS` et `HEAD` possèdent des fonctionnalités particulières que nous aborderons ultérieurement.

Les méthodes HTTP confèrent une signification aux requêtes transmises au serveur et servent de fondement pour mettre en œuvre des opérations couramment désignées sous l'acronyme CRUD (*Create, Read, Update, Delete*) :

  - *Create* (créer).
  - *Read* (lire).
  - *Update* (mettre à jour)
  - *Delete* (supprimer).

Voici un résumé de la manière dont les méthodes HTTP sont couramment employées pour mettre en œuvre des opérations CRUD :

- **`GET`** : Cette méthode est employée pour récupérer une ressource depuis un serveur. Lorsqu'une requête `GET` est adressée pour obtenir une ressource, le serveur génère une représentation de la ressource demandée, souvent au format JSON. En d'autres termes, `GET` est utilisée pour effectuer des opérations de lecture.

- **`POST`** : La requête `POST` sert à créer une nouvelle ressource sur un serveur. Les données nécessaires à la création de la nouvelle ressource dans la base de données sont transmises dans le corps de la requête HTTP. Le serveur répond à la requête `POST` en indiquant le succès ou l'échec de l'opération. En somme, `POST` est utilisée pour effectuer des opérations de création.

- **`PUT`/`PATCH`** : Les requêtes HTTP de type `PUT` ou `PATCH` sont généralement employées pour mettre à jour une ressource sur un serveur. Lorsqu'une requête `PUT` ou `PATCH` est reçue, le serveur met à jour une ressource existante et informe le client du résultat de l'opération. Par conséquent, `PUT` ou `PATCH` sont utilisés pour effectuer des opérations de mise à jour.

- **`DELETE`** : Cette méthode HTTP est utilisée pour supprimer une ressource sur un serveur. Lorsqu'une requête `DELETE` est reçue, le serveur retire la ressource de la base de données et signale au client le succès ou l'échec de l'opération. En résumé, `DELETE` est utilisée pour effectuer des opérations de suppression.

Bien que non essentielles pour les opérations CRUD, les méthodes `OPTIONS` et `HEAD` proposent les fonctionnalités suivantes :

- **`HEAD`** : La méthode HTTP `HEAD` est couramment employée pour vérifier si une ressource a été modifiée. Lorsqu'un serveur reçoit une requête `HEAD`, il renvoie les en-têtes de la réponse HTTP, toutefois, le corps de la réponse est omis. Les en-têtes sont utilisés par le client pour déterminer si une ressource a subi des modifications. Un exemple de réponse est illustré ci-dessous :
```
HTTP/1.1 200 OK
X-Powered-By: Express
Vary: Origin, Accept-Encoding
Access-Control-Allow-Credentials: true
Cache-Control: no-cache
Pragma: no-cache
Expires: -1
X-Content-Type-Options: nosniff
Content-Type: application/json; charset=utf-8
Content-Length: 169
ETag: W/"a9-hFaXT0CwEV/UcaEmhlcSOAZO47Q"
Date: Sun, 06 Mar 2022 22:06:42 GMT
```

- **`OPTIONS`** : Cette méthode est fréquemment employée pour obtenir des informations sur un point de terminaison de l'API, notamment les méthodes prises en charge. Une réponse à une requête de type `OPTIONS` n'inclut généralement pas de corps. Les en-têtes sont utilisés pour transmettre des informations pertinentes au client. Voici ci-dessous un exemple :
```
HTTP/1.1 204 No Content
X-Powered-By: Express
Vary: Origin, Access-Control-Request-Headers
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET,HEAD,PUT,PATCH,POST,DELETE
Content-Length: 0
Date: Sun, 06 Mar 2022 22:07:18 GMT
Connection: keep-alive
```

## Codes d'état HTTP et messages d'erreur

Une API REST peut retourner des messages d'erreur qui n'apparaissent que lorsque quelque chose ne va pas avec la demande.

Les codes d'état HTTP (de `100+` à `500+`) permettent d'indiquer rapidement l'état de la réponse.

En général, les nombres suivent les règles suivantes :

* `200+` signifie que la requête a réussi.

* `300+` signifie que la requête est redirigée vers une autre URL.

* `400+` signifie qu'une erreur provenant du client s'est produite.

* `500+` signifie qu'une erreur provenant du serveur s'est produite.

Par exemple, si on demande une ressource qui n'existe pas sur le serveur, ce dernier enverra une réponse `404 Not Found`:

# Conception d'une API REST avec Express
-----------------

La conception d'une API REST avec Express implique une utilisation et organisation des routes, des modèles de données et des contrôleurs.

* Les routeurs (*routers*) permettent de définir les itinéraires et les points d'accès de l'API. Ils dirigent les requêtes HTTP vers les contrôleurs appropriés, qui contiennent la logique de gestion des demandes.

* Les modèles (*models*) représentent la structure des données et leur interaction avec la base de données.

* Les contrôleurs (*controllers*) sont responsables de l'exécution des opérations sur les données en fonction des demandes entrantes, puis ils renvoient les réponses correspondantes.

Cette approche modulaire facilite la maintenance, l'extensibilité et la collaboration lors du développement d'une API REST avec Express. Elle permet également de séparer clairement les préoccupations liées aux routes, à la logique métier et aux données, améliorant ainsi la lisibilité et la gestion du code source.

**Remarques :** Dans l'API que nous allons concevoir dans ce cahier, nous n'irons pas utiliser une base de données réelle. Les bases de données seront étudiées à partir du prochain cours. Dans ce cahier les modèles utilisent des `Map` pour stocker les données en mémoire côté serveur.


In [ ]:
"""
Pour modifier le répertoire de travail de Colab et
créer quelques dossiers pour mieux structurer le projet.
"""
%cd /content/express-mock-api/
!mkdir -p models controllers routers

/content/express-mock-api


## Initialisation du projet

Pour démarrer le projet, il est nécessaire de l'initialiser en utilisant la commande `npm init`, qui créera le fichier `package.json` qui contient des informations sur le projet. Une fois le projet initialisé, nous devons installer les dépendances nécessaires en utilisant la commande `npm install`.

In [ ]:
!npm init -y

Wrote to /content/express-mock-api/package.json:

{
  "name": "express-mock-api",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}




In [ ]:
!npm install express @faker-js/faker ejs --save


added 79 packages, and audited 80 packages in 10s

14 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
npm notice 
npm notice New major version of npm available! 9.8.1 -> 10.2.3
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.2.3
npm notice Run npm install -g npm@10.2.3 to update!
npm notice 


## Les modèles

Dans le contexte d'une API, les modèles définissent la structure des données que l'API gère. Ils agissent comme des modèles conceptuels pour les informations que l'API stocke, récupère et manipule. Chaque modèle correspond à un type spécifique d'entité, avec des propriétés et des attributs spécifiques.

Dans le cadre de notre API, nous allons proposer deux modèles : `users` et `products`.

* Le modèle `users` inclut des propriétés telles que le nom, l'adresse, et le numéro de téléphone.
* Le modèle `products` comprend des informations telles que le nom du produit, la description et le prix.

Pour générer des données aléatoires pour ces modèles, nous allons utiliser une bibliothèque [Faker.js](https://fakerjs.dev/). Faker.js est une bibliothèque JavaScript qui permet de créer des données fictives, telles que des noms, des adresses, des dates, etc.

### Le modèle `users` :

In [ ]:
%%javascript --target=disk --filename=models/users.js

const { fakerFR_CA: faker } = require('@faker-js/faker');

faker.seed(525);
const numberUsers = 15;

const createUsers = (nUsers) => {

    const users = new Map();
    for (let i = 0; i < nUsers; i++) {
        const gender = faker.person.sexType();
        const user = {
            _id: faker.string.uuid(),
            firstName: faker.person.firstName(gender),
            lastName: faker.person.lastName(gender),
            address: faker.location.street(),
            city: faker.location.city(),
            province: faker.location.state({abbreviated: true}),
            birthday: faker.date.birthdate().toISOString().split('T').shift(),
            phone: faker.phone.number()
        }
        users.set(user._id, user);
    }
    return users;
}

class UserModel {
    constructor() {
        this.data = createUsers(numberUsers);
    }

    addUser(user) {
        user._id = faker.string.uuid();
        this.data.set(user._id, user);
        return user;
    }

    find(userId) {
        return this.data.get(userId);
    }

    allUsers() {
        return Object.values(Object.fromEntries(this.data));
    }
}

module.exports = new UserModel();

### Le modèle `products` :

In [ ]:
%%javascript --target=disk --filename=models/products.js

const { fakerFR_CA: faker } = require('@faker-js/faker');

faker.seed(525);
const numberProducts = 50;

const createProducts = (nProducts) => {
    const products = new Map();
    for (let i = 0; i < nProducts; i++) {
        const prod = {
            _id: faker.string.uuid(),
            name: faker.commerce.productName(),
            description: faker.commerce.productDescription(),
            price: faker.commerce.price({min: 10, max: 200, dec: 2, symbol: 'C$'})
        }
        products.set(prod._id, prod);
    }
    return products;
}

class ProductModel {
    constructor() {
        this.data = createProducts(numberProducts);
    }

    addProduct(prod) {
        prod._id = faker.string.uuid();
        this.data.set(prod._id, prod);
        return prod;
    }

    find(prodId) {
        return this.data.get(prodId);
    }

    allProducts() {
        return Object.fromEntries(this.data);
    }
}

module.exports = new ProductModel();

## Les contrôleurs

Les contrôleurs sont des composants qui gèrent la logique métier de l'API, en déterminant comment répondre aux différentes requêtes entrantes. Ils agissent comme des intermédiaires entre les routes de l'API et les modèles de données.

Lorsqu'une requête HTTP arrive sur une route particulière, le contrôleur associé est responsable de l'exécution des actions nécessaires pour répondre à cette requête. Cette logique peut inclure la validation des données, l'accès à la base de données, la modification des données, ou la préparation de la réponse HTTP.

### Module utilitaire aux contrôleurs :

In [ ]:
%%javascript --target=disk --filename=controllers/util.js

class Response {

    constructor(code, message, data) {
        this.code = code;
        this.message = message;
        this.data = data;
    }

    static ok(data) {
        return new Response(200, "OK", data);
    }

    static error(code, message, data) {
        return new Response(code, message, data);
    }
}

module.exports = { Response }

### Le contrôleur pour les produits :

In [ ]:
%%javascript --target=disk --filename=controllers/products.js

const model = require("../models/products");
const { Response } = require("./util");

async function allProducts(req, res){
    res.json(Response.ok(model.allProducts()));
}

async function addProduct(req, res){
    const prod = model.addProduct(req.body);
    if (prod) {
        res.json(Response.ok(prod));
    } else {
        res.status(505);
        res.json(Response.error(505, "Erreur d'ajout de produit"));
    }
}

async function findProduct(req, res) {
    let prod = model.find(req.params.prodId);
    if (prod === undefined) {
        res.status(404);
        res.json(Response.error(404, "Produit pas trouvé."));
    } else {
        res.json(Response.ok(prod));
    }
}

module.exports = { allProducts, addProduct, findProduct };

### Le contrôleur pour les utilisateurs :

In [ ]:
%%javascript --target=disk --filename=controllers/users.js

const model = require("../models/users");
const { Response } = require("./util");

async function allUsers(req, res){
    res.json(Response.ok(model.allUsers()));
}

async function addUser(req, res){
    const user = model.addUser(req.body);
    if (user) {
        res.json(Response.ok(user));
    } else {
        res.status(505);
        res.json(Response.error(505, "Erreur d'ajout d'utilisateur"));
    }
}

async function findUser(req, res) {
    let user = model.find(req.params.userId);
    if (user === undefined) {
        res.status(404);
        res.json(Response.error(404, "Utilisateur pas trouvé."));
    } else {
        res.json(Response.ok(user));
    }
}

module.exports = { allUsers, addUser, findUser };

## Les routeurs

Les routeurs agissent comme des gestionnaires de flux de trafic, dirigeant les requêtes HTTP vers les destinations appropriées en fonction de l'URI (*Uniform Resource Identifier*) demandée.

Les routeurs déterminent quelles actions ou contrôleurs doivent être appelés pour répondre à une requête spécifique. Ils permettent de diviser l'application en segments logiques et de définir des groupes de routes en fonction de leur contexte et de leur fonction.

### Le routeur pour les utilisateurs :

In [ ]:
%%javascript --target=disk --filename=routers/users.js

const { Router } = require("express");
const controller = require("../controllers/users");

const router = Router();

router.route("/api/users")
    .get(controller.allUsers)
    .post(controller.addUser);

router.route("/api/users/:userId")
    .get(controller.findUser);

module.exports = router;

### Le routeur pour les produits :

In [ ]:
%%javascript --target=disk --filename=routers/products.js

const { Router } = require("express");
const controller = require("../controllers/products");

const router = Router();

router.route("/api/products")
    .get(controller.allProducts)
    .post(controller.addProduct);

router.route("/api/products/:prodId")
    .get(controller.findProduct);

module.exports = router;

## L'application principale

Finalement, le fichier `index.js` fera l'intégration des routeurs pour diriger le trafic des requêtes. Le fichier démarre l'instance d'Express, puis relie les routeurs pour gérer les différentes parties de l'application.

In [ ]:
%%javascript --target=node --filename=index.js --port=3000

const express = require('express');
const prodRouter = require("./routers/products");
const userRouter = require("./routers/users");

const PORT = process.env.NODE_PORT || 3000;

const app = express();

app.use(express.json());
app.use(prodRouter);
app.use(userRouter);

app.listen(PORT, () => {
    console.log(`Serveur écoutant sur le port ${PORT}`)
});

Serveur écoutant sur le port 3000


## Tester l'API REST à l'aide de `curl`

Une fois le serveur démarré, nous pouvons utiliser `curl` pour tester notre API.

Pour obtenir la liste des utilisateurs :

In [ ]:
!curl -s http://localhost:3000/api/users | json_pp

{
   "code" : 200,
   "data" : [
      {
         "_id" : "f6cc455f-7b8d-4034-b91b-473d280c7c83",
         "address" : "Meunier de la Paix",
         "birthday" : "1982-04-13",
         "city" : "Rollandfield",
         "firstName" : "Armin",
         "lastName" : "Julien",
         "phone" : "653 565-9618",
         "province" : "MB"
      },
      {
         "_id" : "72dcd569-e046-46f7-975a-ae1c7d94e18a",
         "address" : "Blanc des Francs-Bourgeois",
         "birthday" : "1981-09-04",
         "city" : "Rouxfort",
         "firstName" : "Agnès",
         "lastName" : "Boyer",
         "phone" : "614 609-9925",
         "province" : "NL"
      },
      {
         "_id" : "cac5f8d8-0d8b-41ee-9c40-5e2c710fb5bd",
         "address" : "Élisée Pastourelle",
         "birthday" : "1977-02-19",
         "city" : "Robertside",
         "firstName" : "Alphonsine",
         "lastName" : "Marchand",
         "phone" : "388 476-5471, poste 069",
         "province" : "SK"
      },
      {
 

Pour obtenir des informations sur l'utilsateur dont l ídentifiant est `a75c3569-d6db-4f48-a76e-6c02e0116cce` :

In [ ]:
!curl -s http://localhost:3000/api/users/ec36be7f-ae05-42be-a0c0-ff18d42d7096 | json_pp

{
   "code" : 200,
   "data" : {
      "_id" : "ec36be7f-ae05-42be-a0c0-ff18d42d7096",
      "address" : "Althée des Grands Augustins",
      "birthday" : "1943-05-15",
      "city" : "Fort Huguetteland",
      "firstName" : "Cyprien",
      "lastName" : "Richard",
      "phone" : "704 397-3287",
      "province" : "AB"
   },
   "message" : "OK"
}


Pour obtenir la liste des produits :

In [ ]:
!curl -s http://localhost:3000/api/products | json_pp

## Activité en classe
------------

Dans l'exemple d'API REST que nous avons présenté, vous devez implémenter les routes pour la mise à jour et suppression de produits et d'utilisateurs. Vous aurez à mettre à jour les routeurs, les contrôleurs et les modèles pour permettre cette mise à jour.

**Solution disponible avec l'exemple complet sur l'entrepôt git du cours**

# JSON Server pour la conception d'une API REST factice
---------

[JSON Server](https://github.com/typicode/json-server) est un outil pour créer des API factices (*Mock API*) qui utilisent des fichiers JSON comme source de données. C'est une solution simple et légère qui permet de créer rapidement une API simulée à partir de données stockées dans des fichiers JSON, sans nécessiter de configuration de serveur complexe ni de gestion de base de données.

Pour utiliser JSON Server, il vous suffit d'installer l'outil via npm et de créer un fichier JSON qui contiendra les données de référence.

In [ ]:
"""
Pour modifier le répertoire de travail de Colab.
"""
%cd /content/json-mock-api/

/content/json-mock-api


Nous allons simplemente créer le fichier `package.json` manuellement et installer les dépendances :

In [ ]:
%%writefile package.json
{
  "name": "json-mock-api",
  "version": "1.0.0",
  "description": "Example d'API de simulation avec JSONServer",
  "author": "Marcos Dias de Assuncao",
  "license": "MIT",
  "dependencies": {
    "@faker-js/faker": "^8.2.0",
    "json-server": "^0.17.4",
    "lodash": "^4.17.21"
  }
}

Writing package.json


In [ ]:
!npm install


added 117 packages, and audited 118 packages in 13s

16 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
npm notice 
npm notice New major version of npm available! 9.8.1 -> 10.2.4
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.2.4
npm notice Run npm install -g npm@10.2.4 to update!
npm notice 


## Création du fichier source des données

JSON server utilise un fichier JSON qui sert comme base de données. Le code de la cellule ci-dessous créera un fichier JSON qui contiendra essentiellement les informations sur les utilisateurs et des produits que nous avons vues dans la version Express de notre API.

In [ ]:
%%javascript --target=node --filename=create_db.js

const { fakerFR_CA: faker } = require('@faker-js/faker');
const _ = require('lodash');
const fs = require("fs");

faker.seed(525);
faker.setDefaultRefDate('2023-09-01T00:00:00.000Z');
const numberUsers = 15;
const numberProducts = 50;
const databaseFile = 'db.json';

const createUser = () => {
    const gender = faker.person.sexType();
    return {
        id: faker.string.uuid(),
        firstName: faker.person.firstName(gender),
        lastName: faker.person.lastName(gender),
        address: faker.location.street(),
        city: faker.location.city(),
        province: faker.location.state({abbreviated: true}),
        birthday: faker.date.birthdate().toISOString().split('T').shift(),
        phone: faker.phone.number()
    }
}

const createProduct = () => {
    return {
        id: faker.string.uuid(),
        name: faker.commerce.productName(),
        description: faker.commerce.productDescription(),
        price: faker.commerce.price({min: 10, max: 200, dec: 2, symbol: 'C$'})
    }
}

try {
    console.log(`En train de créer le fichier ${databaseFile}`);
    const users = _.times(numberUsers, createUser);
    const products = _.times(numberProducts, createProduct);
    fs.writeFileSync(databaseFile, JSON.stringify({ users, products }, null, 2));
    console.log(`Fichier ${databaseFile} crée.`);
} catch(err) {
    console.error(`Erreur dans la création de la BD: ${err.message}`);
}

En train de créer le fichier db.json
Fichier db.json crée.


In [ ]:
!head -n 30 db.json

{
  "users": [
    {
      "id": "f6cc455f-7b8d-4034-b91b-473d280c7c83",
      "firstName": "Armin",
      "lastName": "Julien",
      "address": "Meunier de la Paix",
      "city": "Rollandfield",
      "province": "MB",
      "birthday": "1982-04-13",
      "phone": "653 565-9618"
    },
    {
      "id": "72dcd569-e046-46f7-975a-ae1c7d94e18a",
      "firstName": "Agnès",
      "lastName": "Boyer",
      "address": "Blanc des Francs-Bourgeois",
      "city": "Rouxfort",
      "province": "NL",
      "birthday": "1981-09-04",
      "phone": "614 609-9925"
    },
    {
      "id": "cac5f8d8-0d8b-41ee-9c40-5e2c710fb5bd",
      "firstName": "Alphonsine",
      "lastName": "Marchand",
      "address": "Élisée Pastourelle",
      "city": "Robertside",
      "province": "SK",
      "birthday": "1977-02-19",


Maintenant nous pouvons démarrer JSONServer qui créera l'API REST selon les ressources spécifiées dans le fichier `db.json`:

In [ ]:
%%javascript --target=node --filename=index.js --port=3000

const jsonServer = require('json-server');

const port = process.env.NODE_PORT || 3000;
const server = jsonServer.create();
const router = jsonServer.router("./db.json");
const middlewares = jsonServer.defaults();

server.use(middlewares);
server.use(router);

server.listen(port, () => {
    console.log('JSON Server écoute le port ' + port);
})

JSON Server écoute le port 3000


## Tester l'API REST

Nous pouvons utiliser `curl` pour tester l'API, de la même façon que nous l'avons utilisé pour la version Express de l'API.

In [ ]:
!curl -s http://localhost:3000/users?_limit=3

GET /users?_limit=3 200 7.928 ms - 792
[
  {
    "id": "f6cc455f-7b8d-4034-b91b-473d280c7c83",
    "firstName": "Armin",
    "lastName": "Julien",
    "address": "Meunier de la Paix",
    "city": "Rollandfield",
    "province": "MB",
    "birthday": "1982-04-13",
    "phone": "653 565-9618"
  },
  {
    "id": "72dcd569-e046-46f7-975a-ae1c7d94e18a",
    "firstName": "Agnès",
    "lastName": "Boyer",
    "address": "Blanc des Francs-Bourgeois",
    "city": "Rouxfort",
    "province": "NL",
    "birthday": "1981-09-04",
    "phone": "614 609-9925"
  },
  {
    "id": "cac5f8d8-0d8b-41ee-9c40-5e2c710fb5bd",
    "firstName": "Alphonsine",
    "lastName": "Marchand",
    "address": "Élisée Pastourelle",
    "city": "Robertside",
    "province": "SK",
    "birthday": "1977-02-19",
    "phone": "388 476-5471, poste 069"
  }
]

Pour obtenir les informations de l'utilisateur dont l'identifiant est `f6cc455f-7b8d-4034-b91b-473d280c7c83` :

In [ ]:
!curl -s http://localhost:3000/users/f6cc455f-7b8d-4034-b91b-473d280c7c83

GET /users/f6cc455f-7b8d-4034-b91b-473d280c7c83 200 6.034 ms - 234
{
  "id": "f6cc455f-7b8d-4034-b91b-473d280c7c83",
  "firstName": "Armin",
  "lastName": "Julien",
  "address": "Meunier de la Paix",
  "city": "Rollandfield",
  "province": "MB",
  "birthday": "1982-04-13",
  "phone": "653 565-9618"
}

Pour obtenir les trois premiers produits:

In [ ]:
!curl -s http://localhost:3000/products?_limit=3

GET /products?_limit=3 200 2.944 ms - 1185
[
  {
    "id": "4a75c356-9d6d-4bf4-b876-e6c02e0116cc",
    "name": "Artisanal Granit Savon",
    "description": "Ce magnifique radio-phono comprend un excellent récepteur radioL'ensemble, dans une belle ébénisterie teintée palissandre, forme un très phonique 6 lampes et un tourne-disque de vitesses placé sous le couvercle. Beau meuble. Ce modèle, dont le montage particulièrement soigne assure un très bon rendement aussi bien en radio qu'en phono, est garanti mn an.",
    "price": "C$120.00"
  },
  {
    "id": "e171d1a0-5326-46aa-90dd-e4e8382fdd5b",
    "name": "Petit Granit Clavier",
    "description": "Ces médailles et épingles sont en argent avec patine artistique. Elles ont été composées et frappées spécialement dans les ateliers de l'État pour la Société l'Hirondelle.",
    "price": "C$145.00"
  },
  {
    "id": "7b5afe96-a4e6-4a8b-9b7a-84e019bbf3c9",
    "name": "Luxueux Plastique Boule",
    "description": "Carrosserie en tôle d'acier l

Pour envoyer une requête `OPTIONS` pour obtenir et afficher les en-têtes qui contiennent des informations sur quelles opérations sont possibles pour la ressource `/users`:

In [ ]:
!curl -s -I -X OPTIONS http://localhost:3000/users

HTTP/1.1 204 No Content
X-Powered-By: Express
Vary: Origin, Access-Control-Request-Headers
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET,HEAD,PUT,PATCH,POST,DELETE
Content-Length: 0
Date: Tue, 07 Nov 2023 01:30:30 GMT
Connection: keep-alive
Keep-Alive: timeout=5



Pour ajouter un nouvel utilisateur, il nous faut:

- Émettre une requête HTTP type `POST`
- Fournir l'en-tête `Content-Type: application/json` pour indiquer que le contenu du corps de la requête contient un objet JSON.
- Fournir l'entrée à ajouter dans le format JSON.

Le serveur retournera l'entrée ajouté si la requête a été traitée correctement.

In [ ]:
%%bash

curl -s -X POST http://localhost:3000/users \
-H "Content-Type: application/json" \
--data '{ "firstName": "Nicolette",
          "lastName": "Pierre",
          "address": "2536 Sherbrooke Ouest",
          "city": "Montreal",
          "province": "QC",
          "birthday": "1987-02-13T15:45:59.924Z",
          "phone": "514-321-1234" }'

POST /users 201 14.943 ms - 222
{
  "firstName": "Nicolette",
  "lastName": "Pierre",
  "address": "2536 Sherbrooke Ouest",
  "city": "Montreal",
  "province": "QC",
  "birthday": "1987-02-13T15:45:59.924Z",
  "phone": "514-321-1234",
  "id": "fLIXBfK"
}

Une nouvelle entrée a été créée pour l'utilisateur `Nicolette Pierre`.

Supposons maintenant que nous voulons changer l'adresse de `Nicolette Pierre` pour `2717 Ste. Catherine Ouest`.
Nous devons envoyer une requête type `PUT` ou `PATCH` à `users/id-de-nicolette`.
La requête HTTP contiendra l'objet JSON que le serveur utilisera pour mettre à jour l'entrée `users/id-de-nicolette`. La requête ressemble à celle utilisée pour ajouter une entrée, et le serveur retournera l'entrée mise à jour si la requête a été traitée correctement.


In [ ]:
%%bash
curl -s -X PATCH http://localhost:3000/users/fLIXBfK \
-H "Content-Type: application/json" \
--data '{ "address": "2717 Ste. Catherine Ouest" }'

PATCH /users/fLIXBfK 200 6.625 ms - 226
{
  "firstName": "Nicolette",
  "lastName": "Pierre",
  "address": "2717 Ste. Catherine Ouest",
  "city": "Montreal",
  "province": "QC",
  "birthday": "1987-02-13T15:45:59.924Z",
  "phone": "514-321-1234",
  "id": "fLIXBfK"
}

Pour accéder au serveur sur autre onglet du navigateur (disponible sur Chrome) :

In [ ]:
from google.colab import output
print("Cliquez sur le lien pour accéder le serveur:")
print(output.eval_js(f"google.colab.kernel.proxyPort({3000})"))

# Activité en classe
-----

## Créer des entrées

* Vous devez émettre une requête à l'aide de `curl` pour créer un nouveau produit. Ensuite, vous devez créer une requête HTTP pour mettre à jour le la description du produit crée.

<!--

!curl -s -X POST http://localhost:3000/products/ -H "Content-Type: application/json" --data '{"name": "Cafetière automatique", "description": "Une cafetière automatique avec moulin intégré pour des cafés fraîchement moulus à chaque tasse.", "price": "C$129.99"}'

-->

<!--

%%writefile patch.json
{"description": "Dotée d'une fonction programmable et d'une carafe en verre, cette caferière vous offre une expérience caféinée agréable à tout moment de la journée."}

curl -s -X PATCH http://localhost:3000/products/id-du-produit-cree -H "Content-Type: application/json" --data @patch.json

-->

In [ ]:
%%writefile produit.json
{
   "name": "Cafetière automatique",
   "description": "Une cafetière automatique avec moulin intégré pour des cafés fraîchement moulus à chaque tasse.",
   "price": "C$129.99"
}

Writing produit.json


In [ ]:
!curl -s -X POST http://localhost:3000/products/ -H "Content-Type: application/json" --data @produit.json | json_pp

POST /products/ 201 24.002 ms - 202
{
   "description" : "Une cafetière automatique avec moulin intégré pour des cafés fraîchement moulus à chaque tasse.",
   "id" : "51WydK0",
   "name" : "Cafetière automatique",
   "price" : "C$129.99"
}


In [ ]:
%%writefile patch.json
{"description": "Dotée d'une fonction programmable et d'une carafe en verre, cette caferière vous offre une expérience caféinée agréable à tout moment de la journée."}


Writing patch.json


In [ ]:
!curl -s -X PATCH http://localhost:3000/products/51WydK0 -H "Content-Type: application/json" --data @patch.json

PATCH /products/51WydK0 200 5.065 ms - 257
{
  "name": "Cafetière automatique",
  "description": "Dotée d'une fonction programmable et d'une carafe en verre, cette caferière vous offre une expérience caféinée agréable à tout moment de la journée.",
  "price": "C$129.99",
  "id": "51WydK0"
}

In [ ]:
!curl -s http://localhost:3000/products/51WydK0

GET /products/51WydK0 200 3.823 ms - 257
{
  "name": "Cafetière automatique",
  "description": "Dotée d'une fonction programmable et d'une carafe en verre, cette caferière vous offre une expérience caféinée agréable à tout moment de la journée.",
  "price": "C$129.99",
  "id": "51WydK0"
}

# Références
------------

* [Redhat - Une API REST, qu'est-ce que c'est ?](https://www.redhat.com/fr/topics/api/what-is-a-rest-api)
* [REST Architectural Constraints](https://restfulapi.net/rest-architectural-constraints/)
* [Understanding And Using REST APIs](https://www.smashingmagazine.com/2018/01/understanding-using-rest-api/)
* Thomas Hunter, Distributed Systems with Node.js: Building Enterprise-Ready Backend Services, November, 2020.
* [MDN - En-têtes HTTP](https://developer.mozilla.org/fr/docs/Web/HTTP/Headers)
* [Command line tool and library for transferring data with URLs](https://curl.se/)